## Import libraries

In [1]:
import os
import json

from llama_index.core import (
    VectorStoreIndex,
    Settings,
    Document,
    StorageContext,
)
from llama_index.vector_stores.chroma import ChromaVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.ollama import Ollama

from llama_index.core.chat_engine.types import ChatMode
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core import ChatPromptTemplate

from prompts import SYSTEM_PROMPT, USER_PROMPT

## LLM + Embeddings

In [2]:
llm = Ollama(
    model='llama_test',
    )

lc_embed_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-small"
)
embed_model = LangchainEmbedding(lc_embed_model)

Settings.llm = llm
Settings.embed_model = embed_model

## Vector store

In [3]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="../chromadb")
chroma_collection = chroma_client.get_collection("rag_database")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

# Query Pipeline

## Tạo QA Template

In [4]:
chat_text_qa_msgs = [
    ("system", SYSTEM_PROMPT),
    ("user", USER_PROMPT),
]
text_qa_template = ChatPromptTemplate.from_messages(chat_text_qa_msgs)

## Khởi tạo Chat Engine

In [13]:
memory = ChatMemoryBuffer.from_defaults(token_limit=2048)

chat_engine = index.as_chat_engine(
    similarity_top_k=4,
    response_mode="compact",
    chat_mode=ChatMode.CONTEXT,
    text_qa_template=text_qa_template,
    chat_memory=memory,
)

## Hỏi đáp

In [14]:
# Test the RAG pipeline
def ask_question(question):
    response = chat_engine.chat(question)
    print("Q:", question)
    print("\nA:", response)
    print(f"\nTotal Retrieved Nodes: {len(response.source_nodes)}")
    print("\nRetrieved Nodes:")
    
    for i, node in enumerate(response.source_nodes, 1):
        print(f"\nNode #{i}")
        print(f"Score: {node.score:.4f}")
        print(f"ID: {node.metadata.get('id')}")
        print(f"Unit: {node.metadata.get('unit')}")
        print(f"Section: {node.metadata.get('section')}")
        print(f"Type: {node.metadata.get('type', 'N/A')}")
        print(f"Content Preview: {node.text[:200]}...")
        print("-" * 80)

In [16]:
# Example questions
questions = [
    "query: Sự khác nhau giữa thì quá khứ đơn và quá khứ tiếp diễn là gì?",
    "query: Cho ví dụ cụ thể về sự khác nhau giữa thì quá khứ đơn và quá khứ tiếp diễn",
    "query: Những lưu ý khi sử dụng thì quá khứ đơn và quá khứ tiếp diễn",
    "query: Công thức tạo câu phủ định trong thì quá khứ đơn",
    "query: Công thức tạo câu phủ định trong thì quá khứ tiếp diễn",
]

for q in questions:
    ask_question(q)
    print("\n" + "-"*80 + "\n")


Q: query: Sự khác nhau giữa thì quá khứ đơn và quá khứ tiếp diễn là gì?

A: Thì quá khứ đơn dùng để diễn tả hành động xảy ra và kết thúc trong một khoảng thời gian xác định, còn thì quá khứ tiếp diễn dùng để diễn tả hành động xảy ra tại một thời điểm xác định trong quá khứ.

Total Retrieved Nodes: 4

Retrieved Nodes:

Node #1
Score: 0.6760
ID: page_3_1
Unit: None
Section: None
Type: N/A
Content Preview: passage: 

Type: text
LỜI NÓI ĐẦU

Tiếng Anh 12 – Global Success – Sách học sinh được Nhà xuất bản Giáo dục Việt Nam
tổ chức biên soạn theo “Chương trình giáo dục phổ thông: Chương trình môn Tiếng Anh...
--------------------------------------------------------------------------------

Node #2
Score: 0.6662
ID: unit_1_page_14_1
Unit: Life stories we admire
Section: SPEAKING
Type: N/A
Content Preview: passage: Unit: Life stories we admire
Section: SPEAKING
Type: text
2 Work in pairs. A should ask B questions to complete his/her card about Vo Thi Sau. Then B should do the same to complete 